In [4]:
import numpy as np

In [5]:
batch_size = 8196

mnist = np.load("dataset/mnist-original.npy", allow_pickle=True)
X = mnist.item().get("data").T
y = mnist.item().get("label")[0]

model = np.load("weights/model.npy", allow_pickle=True).item()
fc1w = np.array(model['fc1w'])
fc2w = np.array(model['fc2w'])
fc3w = np.array(model['fc3w'])

In [12]:
sign = lambda x: float(1) if x>0 else float(-1)
sign = np.vectorize(sign)
adj = lambda x: x*2-1
adj = np.vectorize(adj)

def quantize(x):
    return 0 if x==1 else 1
def concat4(li, point):
    result = np.array([quantize(li[point])], dtype=np.uint32)[0].astype(np.uint32)
    for k in range(1, 32):
        i = point + k
        result <<= 1
        result &= 0xFFFFFFFF
        result |= quantize(li[i])
        result &= 0xFFFFFFFF
    return result.astype(np.int32)

def pack(A, n):
    A_bit = np.array([0]*(n//32), dtype=np.int32)
    
    A_lin = np.reshape(A, (n,))
    
    for i in range(0, n, 32):
        A_bit[i//32] = concat4(A_lin, i) 
        
    return A_bit

In [13]:
def preprocessWeights(fc1w, fc2w, fc3w):
    fc1w_q = np.array([list(arr)+([0]*16) for arr in sign(fc1w)])
    fc1w_bit = pack(fc1w_q.T.T, fc1w_q.shape[0]*fc1w_q.shape[1])
    fc1w_bit = pack(fc1w_q, fc1w_q.shape[0]*fc1w_q.shape[1])
    fc2w_q = sign(fc2w)
    fc2w_bit = pack(fc2w_q, fc2w_q.shape[0]*fc2w_q.shape[1])
    fc3w_q = sign(fc3w)
    fc3w_bit = pack(fc3w_q, fc3w_q.shape[0]*fc3w_q.shape[1])
    
    return fc1w_bit, fc2w_bit, fc3w_bit

def preprocessModel(X, y, batch_size):
    numpydict = {"X":[],"y":[]}
    for idx in range(len(X)//batch_size):
        X0 = X[batch_size * idx:batch_size * idx + batch_size]
        X0_q = sign(adj(X0))
        X0_q = np.array([list(arr)+[1]*16 for arr in X0_q])
        X0_bit = pack(X0_q, X0_q.shape[0]*X0_q.shape[1])
        Y0 = y[batch_size * idx:batch_size * idx + batch_size]
        numpydict["X"].append(X0_bit)
        numpydict["y"].append(Y0)

    np.save('dataset/mnist-bit_B{}.npy'.format(batch_size), numpydict, allow_pickle=True)

In [ ]:
preprocessModel(X, y, batch_size)